In [33]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta

In [ ]:
#directories

In [29]:
def clean_df(df):
    result = df.loc[(df['Movement'] != "start") & (df['EyeTrackingEnabled'] == True) & (df['EyeCalibrationValid'] == True) & (df['EyeTrackingEnabledAndValid'] == True) & (df['EyeTrackingDataValid'] == True) & (df['GazeInputSource'] == "Eyes")].copy()
    result.reset_index(inplace=True, drop=True)
    return result

In [34]:
def calculate_differences(df):
    cleaned_df = clean_df(df)
    difference_df = cleaned_df.loc[1:, ['Movement', 'Time', 'GazeAngleX', 'GazeAngleY']]
    difference_df.reset_index(inplace=True, drop=True)
    difference_df['euclideanDifference'] = difference_df.apply(lambda row: np.linalg.norm(np.array((row['GazeAngleX'], row['GazeAngleY'])) - np.array((cleaned_df.loc[row.name, 'GazeAngleX'], cleaned_df.loc[row.name, 'GazeAngleY']))), axis=1)
    difference_df['temporalDifference'] = difference_df.apply(lambda row: (datetime.strptime(row['Time'], "%m/%d/%Y %H:%M:%S.%f") - datetime.strptime(cleaned_df.loc[row.name, 'Time'], "%m/%d/%Y %H:%M:%S.%f")).total_seconds(), axis=1)
    return difference_df

In [35]:
df = pd.read_csv("../Data/Study/01/raw/calibration_20230504162033.csv")
print(calculate_differences(df))

        Movement                        Time  GazeAngleX  GazeAngleY  \
0     transition  05/04/2023 16:20:36.064787   -0.377965   -6.150475   
1     transition  05/04/2023 16:20:36.084346   -0.374920   -6.159284   
2     transition  05/04/2023 16:20:36.094506   -0.371671   -6.167336   
3     transition  05/04/2023 16:20:36.114137   -0.368408   -6.175092   
4     transition  05/04/2023 16:20:36.134417   -0.366569   -6.183158   
...          ...                         ...         ...         ...   
1805      static  05/04/2023 16:21:06.165059   17.886810  -18.380950   
1806      static  05/04/2023 16:21:06.184899   17.887510  -18.360830   
1807      static  05/04/2023 16:21:06.194807   17.887200  -18.338110   
1808      static  05/04/2023 16:21:06.215047   17.886730  -18.315500   
1809      static  05/04/2023 16:21:06.235064   17.888830  -18.293520   

      euclideanDifference  temporalDifference  
0                0.009654            0.010264  
1                0.009320            0.